<a href="https://colab.research.google.com/github/arutraj/ML_Basics/blob/main/15_7_Transfer_Learning__Train_some_layers_while_freeze_others.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TensorFlow 1.x selected.


## Transfer Learning: Train some layers while freeze others

1. Import necessary libraries
2. Load the data
3. Pre-Process the data
4. Load weights of pretrained model and freeze few layers
5. Train the model
6. Get predictions

## 1. Import neccessary libraries

Here we will import the required libraries and functions.

In [1]:
# import necessary libraries for plotting images
import numpy as np
import matplotlib.pyplot as plt
# plot in jupyter notebook itself
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Model
from keras.utils import to_categorical
#used to preprocess data according to VGG16
from keras.applications.vgg16 import preprocess_input
#for instantiating the model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer

## 2. Load the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# unzipping the Dataset.zip file
#!unzip /content/drive/My\ Drive/Dataset.zip
!ls /content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/*jpg |wc -l

2352


In [4]:
#reading the csv file containing data labels
data = pd.read_csv('/content/emergency_classification.csv')

In [5]:
# create an empty python list
X = []

# go through all the image locations one by one
for img_name in data.image_names:
    # read the image from location
    img = plt.imread('/content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/' + img_name)
    # pile it one over the other
    X.append(img)

# convert this python list to a single numpy array
X = np.array(X)

In [6]:
#getting the labels for images
y = data.emergency_or_not.values

## 3. Pre-Process the data

Steps :
1. Pre-process the data as per model's requirement
2. Prepare training and validation set

In [7]:
#preprocess input images accordiing to requirements of VGG16 model
X = preprocess_input(X)

In [8]:
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency
# with only one of them is true for every image
y = to_categorical(y)

In [9]:
# splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load weights of pretrained model and freeze few layers

In [10]:
base_model = VGG16(weights='imagenet')

553467096/553467096 [==============================] - 24s 0us/step


In [11]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
# creating our own model
x = Dense(100, activation='relu', name='fc1')(base_model.layers[-4].output)
y = Dense(2, activation='softmax', name='prediction')(x)
my_model = Model(base_model.input, y)
my_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
my_model.layers

In [14]:
len(my_model.layers)

22

In [21]:
# to set the first 15 layers to non-trainable (weights will not be updated) FREEZE the first few layers
for layer in my_model.layers[:16]:
    layer.trainable = False

In [22]:
# compile the model
my_model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

## 5. Training the model

In [23]:
# training the model
my_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
52/52 [==============================] - 12s 210ms/step - loss: 0.6785 - accuracy: 0.5863 - val_loss: 0.6862 - val_accuracy: 0.5609
Epoch 2/10
52/52 [==============================] - 10s 195ms/step - loss: 0.6784 - accuracy: 0.5863 - val_loss: 0.6864 - val_accuracy: 0.5609
Epoch 3/10
52/52 [==============================] - 10s 198ms/step - loss: 0.6784 - accuracy: 0.5863 - val_loss: 0.6866 - val_accuracy: 0.5609
Epoch 4/10
52/52 [==============================] - 10s 197ms/step - loss: 0.6783 - accuracy: 0.5863 - val_loss: 0.6866 - val_accuracy: 0.5609
Epoch 5/10
52/52 [==============================] - 10s 193ms/step - loss: 0.6783 - accuracy: 0.5863 - val_loss: 0.6867 - val_accuracy: 0.5609
Epoch 6/10
52/52 [==============================] - 10s 191ms/step - loss: 0.6783 - accuracy: 0.5863 - val_loss: 0.6868 - val_accuracy: 0.5609
Epoch 7/10
52/52 [==============================] - 10s 190ms/step - loss: 0.6783 - accuracy: 0.5863 - val_loss: 0.6868 - val_accuracy: 0.5609

## 6. Get Predictions

In [24]:
# get predictions
predictions = my_model.predict(X_valid)
predicted_class = np.argmax(predictions,axis=1)

23/23 [==============================] - 3s 125ms/step


In [25]:
# checking the performance of model on validation set
accuracy_score(y_valid[:,1], predicted_class)

0.5609065155807366

## Steps to use transfer learning where we train some layers while freeze others

1. Import necessary libraries
2. Load the data
3. Pre-Process the data
4. Load weights of pretrained model and freeze few layers
5. Train the model
6. Get predictions